In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1200)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1200)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)
print(coordinates)

[(-66.39707932803671, -121.64801806627105), (-89.07053335245237, 179.94240033189607), (-27.645685665122755, -115.42422113225317), (-75.43714812754808, 74.04483260169954), (-85.73157145787137, -173.32955956434864), (45.09625166865632, 160.92842167606568), (-69.50160931919879, -81.57644534201627), (-36.985467802050565, 153.60493623586723), (32.07881018017609, 127.28918984350122), (-29.686836558501746, 64.53854413350695), (-21.45013615044421, 121.71857897631986), (53.978012895497585, -162.826995719297), (74.88636274106938, -128.54786001331803), (-5.989505347218724, -161.73494358231142), (-64.0720010103166, 7.199217741408972), (2.9389660548507948, -48.61580018041647), (55.962383638482606, 11.260817896665372), (0.4256872110180012, -16.10006890921673), (-55.66382771194022, -124.87504861662791), (-89.95059453087778, 166.74368232232274), (-1.989604284226104, -146.13042907499812), (-47.49848638494688, 153.58237919807698), (67.85159374798684, -152.23578871502454), (-83.79527247415545, 155.755016

In [4]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [5]:
# Create a list for holding the city names
cities = []

# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
        
# Print the city count to confirm sufficient count.
#print(cities)
len(cities)

513

In [6]:
# Import requests module
import requests 

# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime

In [7]:
# Import API key from the config.py file
from config import api_key

# URL for OpenWeatherMap with units set to imperail and my API key
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + api_key

In [8]:
# Start the timer
start = time.time()

# Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
    
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

# End the timer and print out time elapsed
end = time.time()
print ("Time elapsed:", end - start)

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | rikitea
Processing Record 2 of Set 1 | kaitangata
Processing Record 3 of Set 1 | saint-philippe
Processing Record 4 of Set 1 | vaini
Processing Record 5 of Set 1 | severo-kurilsk
Processing Record 6 of Set 1 | punta arenas
Processing Record 7 of Set 1 | ulladulla
Processing Record 8 of Set 1 | fukue
Processing Record 9 of Set 1 | mahebourg
Processing Record 10 of Set 1 | port hedland
Processing Record 11 of Set 1 | bethel
Processing Record 12 of Set 1 | tuktoyaktuk
Processing Record 13 of Set 1 | samusu
City not found. Skipping...
Processing Record 14 of Set 1 | hermanus
Processing Record 15 of Set 1 | amapa
Processing Record 16 of Set 1 | asnaes
Processing Record 17 of Set 1 | robertsport
Processing Record 18 of Set 1 | bluff
Processing Record 19 of Set 1 | atuona
Processing Record 20 of Set 1 | hobart
Processing Record 21 of Set 1 | college
Processing Record 22 of Set 1 | nikolskoye
Processing 

Processing Record 36 of Set 4 | bayan
Processing Record 37 of Set 4 | vancouver
Processing Record 38 of Set 4 | ust-kuyga
Processing Record 39 of Set 4 | gainesville
Processing Record 40 of Set 4 | copiapo
Processing Record 41 of Set 4 | petropavl
Processing Record 42 of Set 4 | ulety
Processing Record 43 of Set 4 | desaguadero
Processing Record 44 of Set 4 | luoyang
Processing Record 45 of Set 4 | mujiayingzi
Processing Record 46 of Set 4 | grand river south east
City not found. Skipping...
Processing Record 47 of Set 4 | saldanha
Processing Record 48 of Set 4 | lorengau
Processing Record 49 of Set 4 | saskylakh
Processing Record 50 of Set 4 | nuristan
Processing Record 1 of Set 5 | kamenskoye
City not found. Skipping...
Processing Record 2 of Set 5 | praia da vitoria
Processing Record 3 of Set 5 | sao joao da barra
Processing Record 4 of Set 5 | constitucion
Processing Record 5 of Set 5 | saint-leu
Processing Record 6 of Set 5 | avera
Processing Record 7 of Set 5 | chicama
Processing

Processing Record 25 of Set 8 | attawapiskat
City not found. Skipping...
Processing Record 26 of Set 8 | kyren
Processing Record 27 of Set 8 | alofi
Processing Record 28 of Set 8 | iquitos
Processing Record 29 of Set 8 | pangoa
Processing Record 30 of Set 8 | inyonga
Processing Record 31 of Set 8 | atar
Processing Record 32 of Set 8 | cockburn town
Processing Record 33 of Set 8 | longyearbyen
Processing Record 34 of Set 8 | sisimiut
Processing Record 35 of Set 8 | redmond
Processing Record 36 of Set 8 | minna
Processing Record 37 of Set 8 | san andres
Processing Record 38 of Set 8 | alekseyevsk
Processing Record 39 of Set 8 | nicoya
Processing Record 40 of Set 8 | klaksvik
Processing Record 41 of Set 8 | paramirim
Processing Record 42 of Set 8 | buriti dos lopes
Processing Record 43 of Set 8 | odweyne
Processing Record 44 of Set 8 | axim
Processing Record 45 of Set 8 | bereda
Processing Record 46 of Set 8 | boo
Processing Record 47 of Set 8 | palauig
Processing Record 48 of Set 8 | uri

In [12]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)

# reorder columns
new_col_order = ["City","Country", "Date" , "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df = city_data_df[new_col_order]

# view dataframe
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Rikitea,PF,2022-01-21 21:06:27,-23.1203,-134.9692,78.44,74,100,8.84
1,Kaitangata,NZ,2022-01-21 21:06:27,-46.2817,169.8464,60.93,60,14,9.55
2,Saint-Philippe,RE,2022-01-21 21:06:28,-21.3585,55.7679,81.95,87,62,17.36
3,Vaini,TO,2022-01-21 21:06:28,-21.2000,-175.2000,80.87,71,66,15.19
4,Severo-Kurilsk,RU,2022-01-21 21:02:16,50.6789,156.1250,28.49,82,97,8.77
5,Punta Arenas,CL,2022-01-21 21:01:58,-53.1500,-70.9167,61.70,54,49,11.05
6,Ulladulla,AU,2022-01-21 21:06:12,-35.3500,150.4667,65.88,68,96,2.17
7,Fukue,JP,2022-01-21 21:06:28,32.6881,128.8419,48.45,58,88,3.65
8,Mahebourg,MU,2022-01-21 21:02:15,-20.4081,57.7000,81.25,84,37,18.99
9,Port Hedland,AU,2022-01-21 21:04:40,-20.3167,118.5667,82.45,83,56,5.75


In [14]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"

# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")